<a href="https://colab.research.google.com/github/aliahalotaibi73/week6_exercises/blob/main/Motion_Detection_Practice_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motion Detection Practice Exercise

In this exercise, you will implement a motion detection system using OpenCV. Your task is to detect and track moving objects in a video file. The goal is to identify areas of motion and process each frame accordingly. The video for this exercise can be found under `Example_Motion.mp4`.

## Objectives
- Load and process a video using OpenCV.
- Create and utilize a background subtractor for motion detection.
- Apply the background subtractor to each video frame to identify areas of motion.
- Use OpenCV to manipulate and save the processed video.
- Track and highlight moving objects in each frame.
- Save the processed video with the detected motion areas.

This exercise is designed to help you understand the fundamentals of motion detection and background subtraction in video processing. By the end of this exercise, you should be able to implement a basic motion detection pipeline.

## Importing Required Libraries

In this step, we'll import the essential libraries for processing video and performing motion detection.
Make sure you have `opencv-python` installed in your environment

In [24]:
import numpy as np
import cv2

## Setting Up Video Capture
In this section, you'll set up video capture using OpenCV's `cv2.VideoCapture()` to read the input video file. This is where you initialize the video stream that you'll process frame by frame.

In [25]:
cap = cv2.VideoCapture('/content/Example_Motion.mp4')
print(cap.isOpened())

True


## Defining the Video Writer to Save the Output
Here, you'll define the video writer's object `cv2.VideoWriter()`, which will be used to save the processed video with the motion detection applied.

In [26]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('motion_output.mp4', fourcc, 20.0, (600, 500))

## Creating the Background Subtractor

Motion detection in videos often involves background subtraction.
In this step, we'll create a background subtractor object using OpenCV's `createBackgroundSubtractorMOG2` method.

In [27]:
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

## Processing the Video Frame by Frame

Now, we'll loop through each frame of the video, apply the background subtractor to detect motion,
and then write the processed frame to the output video.

In [28]:
while True:
    # Capture each frame of the video
    success, img = cap.read()

    # Check if the frame is captured successfully
    if success:
        # Resize the frame for consistency
        img = cv2.resize(img, (600, 500))

        # Apply the background subtractor to detect moving objects
        fgmask = fgbg.apply(img)

        # Create a binary thresholded image for better motion detection
        _, thresh = cv2.threshold(fgmask.copy(), 180, 255, cv2.THRESH_BINARY)

        # Define a kernel for morphological operations
        kernel = np.ones((7, 7), np.uint8)

        # Apply erosion to remove noise from the thresholded image
        thresh = cv2.erode(thresh, kernel)

        # Apply dilation to strengthen the detected moving objects
        thresh = cv2.dilate(thresh, None, iterations=6)

        # Find contours of the detected motion
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw rectangles around detected motion
        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)

            # Only consider significant motion (area > 1200)
            if area > 1200:
                # Get the bounding box coordinates for the motion
                x, y, w, h = cv2.boundingRect(contour)

                # Draw a rectangle around the detected motion and label it
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 3)
                cv2.putText(img, 'MOTION DETECTED', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                # Write the processed frame with motion detection to the output video
                out.write(img)

    else:
        # Break the loop if no more frames are available
        break

In [29]:
# while True:
#   success, img = cap.read()
#   if not success:
#     break

#     img = cv2.resize(img, (600, 500))
#     fgmask = fgbg.apply(img)
#     _, thresh = cv2.threshold(fgmask, 180, 255,cv2.THRESH_BINARY)
#      thresh = cv2.dilate(cv2.erode(thresh, np.ones((7, 7), np.uint8)), None, iterations=6)

#       contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

#       for contour in contours:
#         if cv2.contourArea(contour) > 1200:
#           x, y, w, h = cv2.boundingRect(contour)
#           cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0,255), 3)
#           cv2.putText(img, 'MOTION DETECTED', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2) out.write(img)

## Releasing Resources

Once all frames are processed, it's important to release the video capture and writer objects to free up system resources.

* Resource Management: The release() methods are used to properly close and free up resources allocated by OpenCV for video capture and writing.
Window Management: cv2.destroyAllWindows() closes any display windows, ensuring a clean exit without hanging or leaving unwanted windows open.

In [30]:
# Release the video capture and writer objects
cap.release()
out.release()

# Close all OpenCV windows
cv2.destroyAllWindows()